In [36]:

from keras.applications.mobilenet import MobileNet, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np




In [37]:
def preprocessingImages(path, type):
    """
    Input: Path
    Output: Preprocessed images using ImageDataGenerator
    """
    if type == "train":
        image_data = ImageDataGenerator(
            zoom_range=0.2,
            shear_range=0.2,
            preprocessing_function= preprocess_input,
            horizontal_flip=True,
        )
    else:
        # Only preprocess validation/testing data
        image_data = ImageDataGenerator(
            preprocessing_function= preprocess_input
        )
    
    # Ensure images are resized to 224x224
    image = image_data.flow_from_directory(
        directory=path,
        target_size=(224, 224),  # Matches model input size
        batch_size=32,
        class_mode="binary"  # Must match the type of classification
    )
    
    return image


In [38]:
test_data = preprocessingImages("split_dataset/test", "test")

Found 114 images belonging to 2 classes.


In [39]:
# Load the best fit model

model = load_model("bestmodel.keras")

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_data)

print(f"Our model accuracy is {accuracy*100:.2f} %")
print(f"Our model loss is {loss:.4f}")  

4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - accuracy: 0.9913 - loss: 0.0292
Our model accuracy is 99.12 %
Our model loss is 0.0288


In [40]:
def get_prediction(path):
    img = image.load_img(path, target_size=(224, 224))

    input_arr = img_to_array(img)
    input_arr = np.expand_dims(input_arr, axis=0)
    input_arr = input_arr / 255.0

    # Predict with the model
    prob = model.predict(input_arr)[0][0]  # extract scalar float
    # If prob >= 0.5 => class 1 (tumor), else class 0 (healthy)
    pred = 1 if prob >= 0.5 else 0
    return pred

def predict_img(path):

    # Predict with the model
    pred = get_prediction(path)

    if pred == 0:
        print("The MRI image is of a HEALTHY BRAIN")
    else:
        print("The MRI image indicates a TUMOR")
        
        # Display the input image
    plt.imshow(input_arr[0])
    plt.title("input image")
    plt.show()


In [41]:
import os
from tensorflow.keras.preprocessing import image


def evaluate_folder(folder, type):
    image_paths = [
        os.path.join(folder, f) 
        for f in os.listdir(folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]
    h = 1
    type_txt = "TUMOR"
    if type == "no":
        type_txt = "HEALTHY"
        h= 0
    total = 0
    correct = 0
    misclassified = []

    for img_path in image_paths:
        pred = get_prediction(img_path)  # 0 or 1

        total += 1
        if pred == h:
            correct += 1
        else:
            misclassified.append(img_path)

    success_percentage = (correct / total) * 100
    print(f"\nTotal images in '{type}' folder: {total}")
    print(f"Correctly predicted as {type_txt}: {correct}")
    print(f"Success percentage: {success_percentage:.2f}%")

In [42]:
# Go through the yes folder and check

yes_folder = "brain-tumor-mri-dataset/Brain_Tumor_Detection/yes"
evaluate_folder(yes_folder, "yes")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━

In [43]:
# Go through the no folder and check

no_folder = "brain-tumor-mri-dataset/Brain_Tumor_Detection/no"
evaluate_folder(no_folder, "no")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━

In [ ]:
# Download new data from the internet